# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [52]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [55]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vilyuysk,63.7553,121.6247,-26.67,99,68,1.79,RU,1706118360
1,1,mount isa,-20.7333,139.5000,27.87,83,75,0.00,AU,1706118360
2,2,sidrolandia,-20.9319,-54.9614,31.16,39,49,3.65,BR,1706118360
3,3,xilin hot,43.9667,116.0333,-19.18,92,5,3.79,CN,1706118360
4,4,port-aux-francais,-49.3500,70.2167,5.42,70,71,13.83,TF,1706118360


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [56]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["City"]
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
hotel_cities = city_data_df.loc[(city_data_df["Max Temp"]>= 10) & (city_data_df["Max Temp"]<=30)
                              & (city_data_df["Wind Speed"]<= 3) & (city_data_df["Cloudiness"]== 0)]

# Drop any rows with null values
hotel_cities.dropna()

# Display sample data
hotel_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,ushuaia,-54.8000,-68.3000,12.79,54,0,1.54,AR,1706118081
156,156,callao,-12.0667,-77.1500,28.16,61,0,2.57,PE,1706118385
160,160,madalena,41.2162,-8.3345,16.83,82,0,0.72,PT,1706118386
175,175,cascais,38.6979,-9.4215,20.75,70,0,2.06,PT,1706118388
183,183,sant lluis,39.8526,4.2594,15.22,63,0,2.57,ES,1706118390
205,205,tura,25.5198,90.2201,11.83,52,0,2.12,IN,1706118394
211,211,bhinmal,25.0000,72.2500,15.06,24,0,2.20,IN,1706118395
250,250,teguise,29.0605,-13.5640,18.12,49,0,2.57,ES,1706118403
254,254,lazaro cardenas,17.9583,-102.2000,28.73,55,0,2.62,MX,1706118403
295,295,aguilar,37.5148,-4.6572,19.32,57,0,0.71,ES,1706118410


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = hotel_cities.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]] 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,ushuaia,AR,-54.8000,-68.3000,54,
156,callao,PE,-12.0667,-77.1500,61,
160,madalena,PT,41.2162,-8.3345,82,
175,cascais,PT,38.6979,-9.4215,70,
183,sant lluis,ES,39.8526,4.2594,63,
205,tura,IN,25.5198,90.2201,52,
211,bhinmal,IN,25.0000,72.2500,24,
250,teguise,ES,29.0605,-13.5640,49,
254,lazaro cardenas,MX,17.9583,-102.2000,55,
295,aguilar,ES,37.5148,-4.6572,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
callao - nearest hotel: Casa Ronald
madalena - nearest hotel: Hotel de Paredes
cascais - nearest hotel: Cascais Hotel
sant lluis - nearest hotel: Hotel Son Tretze
tura - nearest hotel: No hotel found
bhinmal - nearest hotel: No hotel found
teguise - nearest hotel: Finca Las Laderas
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
aguilar - nearest hotel: Hostal Bellido
baran - nearest hotel: Hotel surya
camarinas - nearest hotel: O Parranda
lata - nearest hotel: Residencial Botânico
bechar - nearest hotel: Hôtel Madania
par naogaon - nearest hotel: Hotel Abokash
hassi messaoud - nearest hotel: AMC
salina cruz - nearest hotel: Hotel Mision San José


,City,Country,Lat,Lng,Humidity,Hotel Name
13,ushuaia,AR,-54.8000,-68.3000,54,Apart Hotel Aires del Beagle
156,callao,PE,-12.0667,-77.1500,61,Casa Ronald
160,madalena,PT,41.2162,-8.3345,82,Hotel de Paredes
175,cascais,PT,38.6979,-9.4215,70,Cascais Hotel
183,sant lluis,ES,39.8526,4.2594,63,Hotel Son Tretze
205,tura,IN,25.5198,90.2201,52,No hotel found
211,bhinmal,IN,25.0000,72.2500,24,No hotel found
250,teguise,ES,29.0605,-13.5640,49,Finca Las Laderas
254,lazaro cardenas,MX,17.9583,-102.2000,55,Hotel Sol del Pacífico
295,aguilar,ES,37.5148,-4.6572,57,Hostal Bellido


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [51]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)